<h1 style='text-aling:center;color:Navy'>  Big Data Systems - Laboratory 1  </h1>

Topics included in this lab:
 1) Map-Reduce (20 minutes)
 2) Semistructured DS: Spanner DB (20 minutes)
 3) Bigtable (20 minutes)

We have allocated 15 minutes for set up and/or to address any other issues. 


This is the first Lab for the Big Data Systems course - Spring 2018.<br>
We are covering the following topics:
<li>Map-Reduce,</li>
<li>Spanner and</li>
<li>Bigtable</li>
<br>
The lab will have an in-class section as well as a homework section. <br>
You need to submit your in-class notebook before the end of the class. For the homework section, please refer to Canvas for the due date.


 

# <span style="color:#3665af">Section #2: Spanner </span><span style="font-size:15px">(Estimated time: 20 minutes) </span>

<hr>
In this section we will practice how to use Google's Spanner database. 
## Pre-reqs:
Your Google cloud account should be ready to deploy services.<br>

### Create a Spanner Instance.

- Create a spanner instace, and annotate the Instance ID. 
- We will use lab1-section2 as instance id in this lab; if you are using another id, you need to change in the connection settings. 
- For this test, configuring one node will suffice. 

### Installing the Python Client. 
We need to have the google-cloud library installed in our system.<br>
If you are using windows/mac, go to the anaconda navigator, then Environments. Click the arrow next to _base (root)_ and select Open terminal. <br>
Then execute the following pip command to install the client library. 

``` bash
$> pip install google-cloud
```

### Getting the Service Account File
As mentioned in the cloud setup instructions, you need to generate a token file so you will be able to connect to spanner. Refer to that document for help. <br>
Here is the summary of the steps. 
1. Go to APIS & Services
2. Go to Credentials
3. Select Create Credentials, and choose Service account key.
4. Select Compute Engine default service account and Key type JSON. Then Create. 
5. Pick the file that was automatically downloaded, and store it in your BSD directory. 


## Hands on...

In [ ]:
# Imports the Google Cloud Client Library.
from google.cloud import spanner

In [ ]:
JSON_SERVICE_KEY = 'path/filename to the google cloud key'

You can learn more about creating instances and using Spanner with Python at [Google Documentation](https://cloud.google.com/spanner/docs/getting-started/python/)


In [ ]:
def explicit():
    ## Function to connect to spanner
    from google.cloud import storage

    # Explicitly use service account credentials by specifying the private key
    # file.
    storage_client = storage.Client.from_service_account_json(JSON_SERVICE_KEY)

    # Make an authenticated API request
    buckets = list(storage_client.list_buckets())
    print(buckets)

You can get help about authentication at [Google Documentation](https://cloud.google.com/docs/authentication/production#auth-cloud-explicit-python)


## Warm up
<hr>

In [ ]:

# Instantiate a client.
spanner_client = spanner.Client()

# Your Cloud Spanner instance ID.
instance_id = 'lab1-section2'

# Get a Cloud Spanner instance by ID.
instance = spanner_client.instance(instance_id)

# Your Cloud Spanner database ID.
database_id = 'lab1-db'           # If you did not create the database already, createit using the cloud console

# Get a Cloud Spanner database by ID.
database = instance.database(database_id)

# Execute a simple SQL statement.
with database.snapshot() as snapshot:
    results = snapshot.execute_sql('SELECT current_date')

    for row in results:
        print(row)

If you didn't recevie an error and the current date was displayed, it means that at this point we're connected to our Spanner database.<rb>

### Let's create a couple of tables
This may take a minute. If you use your google cloud dashboard, you can see the two new tables as you refresh the page.

In [ ]:
operation =database.update_ddl(ddl_statements=[
    """CREATE TABLE Singers (
        SingerId     INT64 NOT NULL,
        FirstName    STRING(1024),
        LastName     STRING(1024),
        SingerInfo   BYTES(MAX)
    ) PRIMARY KEY (SingerId)"""
    ,
    """CREATE TABLE Albums (
        SingerId     INT64 NOT NULL,
        AlbumId      INT64 NOT NULL,
        AlbumTitle   STRING(MAX)
    ) PRIMARY KEY (SingerId, AlbumId),
      INTERLEAVE IN PARENT Singers ON DELETE CASCADE"""
])

# operation = database.create()

print('Waiting for operation to complete...')
operation.result()

print('Created tables in database {} on instance {}'.format( database_id, instance_id))

### QUESTION:
**_In which line the command that creates the database is actually executed on Spanner?_** Explain briefly. 

#### <span style="color:red"> --- Answer HERE --- </span>

### Let's insert some data

In [ ]:

"""Inserts sample data into the given database.
   The database and table must already exist.
"""

with database.batch() as batch:
    batch.insert(
        table='Singers',
        columns=('SingerId', 'FirstName', 'LastName',),
        values=[
            (1, u'Marc', u'Richards'),
            (2, u'Catalina', u'Smith'),
            (3, u'Alice', u'Trentor'),
            (4, u'Lea', u'Martin'),
            (5, u'David', u'Lomond')])
    batch.commit
    
    batch.insert(
        table='Albums',
        columns=('SingerId', 'AlbumId', 'AlbumTitle',),
        values=[
            (1, 1, u'Total Junk'),
            (1, 2, u'Go, Go, Go'),
            (2, 1, u'Green'),
            (2, 2, u'Forever Hold Your Peace'),
            (2, 3, u'Terrified')])
    batch.commit
print('Inserted data.')

### QUESTION:
**_How many statements were executed in Spanner?_** Explain briefly. 

#### <span style="color:red"> --- Answer HERE --- </span>

### Querying the database

In [ ]:
"""Queries sample data from the database using SQL."""


with database.snapshot() as snapshot:
    results = snapshot.execute_sql(
        'SELECT SingerId, AlbumId, AlbumTitle FROM Albums')
    
    display(results)
    
    for row in results:
        display(row)
        print('SingerId: {}, AlbumId: {}, AlbumTitle: {}'.format(*row))
        #print("SingerId: ",row[0],", AlbumId: ",row[1],", AlbumTitle: ",row[2], )

### QUESTIONS:
1. **What kind of object is _results_?**
2. **What kind of object is _row_?**
3. **How is row being accessed?** _**Tip**_: take a look at the commented line.

#### <span style="color:red"> --- Answer HERE --- </span>
1. answer
2. answer
3. answer

In [ ]:
"""Reads sample data from the database."""

with database.snapshot() as snapshot:
    keyset = spanner.KeySet(all_=True)
    results = snapshot.read(
        table='Albums',
        columns=('SingerId', 'AlbumId', 'AlbumTitle',),
        keyset=keyset,)

    for row in results:
        print('SingerId: {}, AlbumId: {}, AlbumTitle: {}'.format(*row))

### QUESTIONS:
**_What is the difference between the first and second part of the code?_**

#### <span style="color:red"> --- Answer HERE --- </span>

## Let's load some interesting data
<hr>

In [ ]:
import pandas as pd
import datetime
from time import time

In [ ]:

# Instantiate a client.
spanner_client = spanner.Client()

# Your Cloud Spanner instance ID.
instance_id = 'lab1-section2'

# Get a Cloud Spanner instance by ID.
instance = spanner_client.instance(instance_id)

# Your Cloud Spanner database ID.
database_id = 'lab1-db2'                             ## Create new Database

# Get a Cloud Spanner database by ID.
database = instance.database(database_id)

# Execute a simple SQL statement.
with database.snapshot() as snapshot:
    results = snapshot.execute_sql('SELECT current_date')

    for row in results:
        print(row)

In [ ]:
start_time=time()
operation =database.update_ddl(ddl_statements=[
    """CREATE TABLE RadiationMeasurements
        (
           CapturedTime timestamp,
           Latitude float64,
           Longitude float64,
           Value float64,
           Unit String(2048),
           Location String(2048),
           DeviceID String(2048),
           MD5Sum String(2048),
           Height String(2048),
           Surface String(2048),
           Radiation String(2048),
           UploadedTime timestamp,
           LoaderID float64
        )PRIMARY KEY (CapturedTime,Latitude,Longitude,Location,UploadedTime,MD5Sum)"""
])

# operation = database.create()

print('Waiting for operation to complete...')
operation.result()

print('Created tables in database {} on instance {}'.format(
    database_id, instance_id))

end_time=time()

print ("run time:",end_time-start_time)

In [ ]:
df_Measures        = pd.read_csv("/path/to/measurements.sample.csv")

In [ ]:
df_Measures.info()

In [ ]:
myData=[]
for index,row in df_Measures[0:5].iterrows():
    aDateTime = datetime.datetime.strptime(row["Captured Time"], '%Y-%m-%d %H:%M:%S')
    display('Row: {}, Timestamp: {} Lat/Lon:{}/{}'.format(index,aDateTime,row["Latitude"],row["Longitude"]))
    myData.append( (index,aDateTime)  )
myData

In [ ]:
#     ****************************************************************
#     TO-DO: Write here a while/for to insert the Primary key data for the first 10 rows. 
#     ****************************************************************

# --Hints:
#     Adapt the code from the sample. 
#     Generate an array inserting tuples (you should use an array and the function append )


### Write your code here.

        

# <span style="color:#5DB664">DELETE YOUR SPANNER INSTANCE AS WHEN YOU FINNISHED </span>

<hr><BR>

# <span style="color:RED">Homework: </span> Yelp Dataset
<hr>
Similar to the example above, we are going to be using the Yelp competition dataset available [here](https://www.kaggle.com/yelp-dataset/yelp-dataset/data). <br>
You should:
1. Download the dataset to your machine.
2. Create a new database.
3. Create the tables necessary, with the proper interleaving configuration to store the business and reviews.
4. Load the yelp_business dataset.
5. Load 10% of the yelp_review dataset.

Hints:
- You probably want to do several batchs. You can do that by using the **[start:end]** operator for the dataframes. 
- You may need to review the documentation for the [Spanner SQL](https://cloud.google.com/spanner/docs/query-syntax)

## Questions:
1. Report the time for loading each dataset.
2. Compare the time to load the rows for the business and the reviews datasets.
3. List the top 10 better/most rated businesses in Colorado. To simplify this, let's assume that the best rated business is the one with the higher sum of ratings. 



In [1]:
## PLACE YOUR CODE STARTING THIS POINT. 

<hr style="border: 3px double navy;" ><br><br><br>